In [116]:
#Write a class (e.g. named UserItemData) in which you will store the read data. 
#I suggest that you have a path argument and three optional arguments in the class constructor: 
 #   from_date (from which date to read data), to_date (by which date to read data), 
  #  and min_ratings (minimum number of ratings a movie should have). Add a method to the 
   # class that tells you how many ratings it has read. I suggest that you add a method that saves the 
    #read data or read the data using the pickle library
import pandas as pd
import datetime

class UserItemData():
    def __init__(self, *args):
        # if args is 1 - path
        if len(args) == 1:
            self.final = pd.read_csv(args[0], sep='\t')
        else:
            self.path = args[0]
            self.from_date = args[1]
            self.to_date = args[2]
            self.min_ratings = args[3]
            self.fd_list = self.from_date.split(".")
            self.td_list = self.to_date.split(".")
            self.inserted_fd = datetime.datetime(int(self.fd_list[2]), int(self.fd_list[1]), int(self.fd_list[0]))
            
            self.df1 = pd.read_csv(self.path, sep='\t')
            #dates applied BAD
            self.cond = self.df1.loc[(self.df1['date_year'] >= int(self.fd_list[2])) & (self.df1['date_year'] <= int(self.td_list[2]))]
            self.df = self.cond.groupby('movieID')["rating"].count().reset_index(name="count")
            self.movies_enough_ratings = self.df["movieID"][self.df["count"] > self.min_ratings]
            self.movies_list = self.movies_enough_ratings.to_list()
            self.final = self.cond[self.cond['movieID'].isin(self.movies_list)]
    def nratings(self):
        return len(self.final.rows)

lolo = UserItemData('data/user_ratedmovies.dat', "12.10.2003", "12.10.2005", 20)

#print(lolo1.df.head(5))
#print(lolo1.cond)
#print(lolo1.td_list[2])
print(lolo.nratings())    


AttributeError: 'DataFrame' object has no attribute 'rows'

In [18]:
#Write a class that reads the movie file and has a get_title(movieID) method that
#returns its title for a given movie ID. Example:

#md = MovieData('data/movies.dat')
#print(md.get_title(1))
#Output:
#Toy story
import pandas as pd
class MovieData():
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(self.path, sep='\t')
    def get_title(self, num):
        return self.df['title'][self.df.id == num]
    
md = MovieData("data/movies.dat")
print(md.get_title(5))
#print(md.get_title(5))

4    Father of the Bride Part II
Name: title, dtype: object


In [3]:
import pandas as pd
import datetime

df = pd.read_csv('data/user_ratedmovies.dat', sep='\t')
print(df.head(5))

   userID  movieID  rating  date_day  date_month  date_year  date_hour  \
0      75        3     1.0        29          10       2006         23   
1      75       32     4.5        29          10       2006         23   
2      75      110     4.0        29          10       2006         23   
3      75      160     2.0        29          10       2006         23   
4      75      163     4.0        29          10       2006         23   

   date_minute  date_second  
0           17           16  
1           23           44  
2           30            8  
3           16           52  
4           29           30  


In [81]:
1 in [1,2]

True